In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

import gm2
import trfp
import plotting_functions as plt2
# import analysis_helper as helper
# import helper_function_candidates as helper_old
# import df_analysis_funcs as df_func
import helper_functions as helper

import field_map_config_run3 as map_config

from IPython.display import clear_output

blinds = np.loadtxt('blinds.txt')
pair_dict = map_config.pair_dict

open_prefix = 'hdf5/2021-04-26_run_'

save_dir = '/data2/aetb/'
save_prefix = str(datetime.date.today())+'_run-3_'


Welcome to JupyROOT 6.22/02


In [3]:
runs = map_config.runs
# runs = ['3g', '3i', '3j', '3k', '3l', '3m', '3n', '3o']

columns = [['st'+str(st)+',dt'] +  ['st'+str(st)+',m'+str(m+1) for m in range(9)] for st in range(72)]
columns = [item for sublist in columns for item in sublist]
sync_offset_df = pd.DataFrame(columns=columns)

for run in runs:
    clear_output()
    
    print 'Run: ' + run + '\n'
    filename = open_prefix + run + '.h5'

    interp_dfs, keys, subrun_df = helper.read_dfs(filename)
    run_pair_dict = pair_dict[run]

    moment_dfs = helper.interp_to_moment(interp_dfs, keys)
    moment_dfs = helper.blind_moments(moment_dfs, keys, blinds)
    corrected_dfs = helper.moment_to_corrected(moment_dfs, keys)
    hybrid_moment_dfs = helper.bloch_style_moments(corrected_dfs, keys)
    baselines_hybrid = helper.station_average(hybrid_moment_dfs, keys)
    hybrid_vtm_dfs = helper.calculate_vtms(hybrid_moment_dfs, keys, baselines_hybrid, run_pair_dict)
 
    ## Sync Offset
    for pair in pair_dict[run]:
        tr_bl_1 = baselines_hybrid['tr'][pair_dict[run][pair][0]]
        tr_bl_2 = baselines_hybrid['tr'][pair_dict[run][pair][1]]
        fp_bl_1 = baselines_hybrid['fp'][pair_dict[run][pair][0]]
        fp_bl_2 = baselines_hybrid['fp'][pair_dict[run][pair][1]]
        time_bl_1 = baselines_hybrid['time'][pair_dict[run][pair][0]]
        time_bl_2 = baselines_hybrid['time'][pair_dict[run][pair][1]]
        
        delta_tr = tr_bl_2 - tr_bl_1
        delta_fp = fp_bl_2 - fp_bl_1
        delta_time = time_bl_2 - time_bl_1
        
        sync_offsets = np.empty((72,9))
        for st in range(72):
            J = helper._choose_J(st)
            sync_offsets[st,0:5] = delta_tr[st,0:5] -  np.matmul(J, delta_fp[st,0:5])
            sync_offsets[st,5:9] = delta_tr[st,5:9]
        
        row = {}
        for st in range(72):
            for m in range(9):
                row['st'+str(st)+',m'+str(m+1)] = sync_offsets[st,m]
            row['st'+str(st)+',dt'] = delta_time[st]

        sync_offset_df = sync_offset_df.append(pd.DataFrame(row, index=['run_'+run+pair[6]]))
    
    
    ## Done w/ sync offsets
    
    for key in hybrid_vtm_dfs.keys():
        save_key = 'run_' + run + key[6]
        print 'Saving ' + save_key
        # save hybrid
        save_path = save_dir + save_prefix + 'hybrid_maps.h5'
        hybrid_vtm_dfs[key].to_hdf(save_path, key=save_key)

print 'Saving Sync Offsets'

save_path = save_dir + save_prefix + 'hybrid_syncs.h5'
save_key = 'sync_offset_df'
sync_offset_df.to_hdf(save_path, key=save_key)

print
print 'Done.'

Run: 3o


Reading fp_df_1

Reading fp_df_2

Reading fp_df_3

Reading tr_df_1

Reading tr_df_2

Reading tr_df_3

Reading tr_df_4

Calculating moments for fp_df_1
Finished calculating all moments for 232494 events.    

Calculating moments for fp_df_2
Finished calculating all moments for 242966 events.     

Calculating moments for fp_df_3
Finished calculating all moments for 199751 events. 

Calculating moments for tr_df_1
Finished calculating all moments for 43 events.

Calculating moments for tr_df_2
Finished calculating all moments for 4430 events.

Calculating moments for tr_df_3
Finished calculating all moments for 4553 events.

Calculating moments for tr_df_4
Finished calculating all moments for 4810 events.

Blinding fp_df_1

Blinding fp_df_2

Blinding fp_df_3

Blinding tr_df_1

Blinding tr_df_2

Blinding tr_df_3

Blinding tr_df_4

Removing trolley footprints for tr_df_1
Removing trolley image from station 0. 
Could not remove footprint for key: tr_df_1

Removing trolley footprin